# 비상장 중소기업의 상장 후 주가 예측 프로젝트

In [1]:
import numpy as np 
import os  
import matplotlib ##그림용
import matplotlib.pyplot as plt ##그림용
import sklearn ##사이킷런
import pandas as pd

np.random.seed(42) ##랜덤시드 

matplotlib.rc('font', family='NanumBarunGothic') ##matplotlib 한글을 위해
plt.rcParams['axes.unicode_minus'] = False ##한글을 위해

필요한 데이터
1. 미시변수 데이터
2. 종가 데이터
3. 거시 데이터
4. 주식수

In [2]:
micro_data = pd.read_csv('datasets//DB//micro_data.csv', engine='python')
closing_data = pd.read_csv('datasets//DB//closing_data.csv', engine='python')
macro_data = pd.read_csv('datasets//DB//macro_data.csv', engine='python')
stock_data = pd.read_csv('datasets//DB//stock.csv', engine='python')

In [3]:
print(micro_data.info())
print(closing_data.info())
print(macro_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47907 entries, 0 to 47906
Data columns (total 42 columns):
회사명                  47907 non-null object
거래소코드                47907 non-null int64
회계년도                 47907 non-null object
산업코드                 47612 non-null float64
산업명                  47612 non-null object
소속코드                 47612 non-null float64
설립일                  47612 non-null object
종업원                  47612 non-null float64
당좌자산(*)              46961 non-null float64
재고자산(*)              46961 non-null float64
비유동자산(*)             46961 non-null float64
유동부채(*)              46961 non-null float64
비유동부채(*)             46961 non-null float64
자본금(*)               46961 non-null float64
자본잉여금(*)             46961 non-null float64
자본조정(*)              46961 non-null float64
기타포괄손익누계액(*)         46961 non-null float64
이익잉여금(*)             46961 non-null float64
매출액(영업수익)(*)         46961 non-null float64
매출원가(*)              46961 non-null float64
매출총이익(손실)        

#### 필요없는 데이터 정제하기
필요없는 데이터의 목록.  
- micro_data
        거래소코드, 산업코드, 기업규모코드, 자본금(*).1, 자본잉여금(*).1, 자본조정(*).1, 기타포괄손익누계액(*).1, 이익잉여금(결손금)(*), 합계
- closing_data
        거래소코드, 종가
- macro_data
        Unnamed: 7, Unnamed: 8, 28번째 객체
- stock_data
        거래소코드, 발행주식 보통주, 발행주식 우선주

In [4]:
micro_data.drop(['거래소코드', '산업코드', '기업규모코드', 
                 '자본금(*).1', '자본잉여금(*).1', '자본조정(*).1', '기타포괄손익누계액(*).1', 
                 '이익잉여금(결손금)(*)', '합계'], axis=1, inplace=True)
closing_data.drop(['거래소코드', '종가'], axis=1, inplace=True)
macro_data.drop(['Unnamed: 7', 'Unnamed: 8'], axis=1, inplace=True)
macro_data.drop([28], axis=0, inplace=True)
stock_data.drop(["거래소코드", "발행주식 보통주", "발행주식 우선주"], axis=1, inplace=True)

### micro_data와 closing_data 연결하기
#### merge column 만들기
미시 변수 데이터와 종가 데이터들을 합치기 위해 회사, 날짜 데이터를 이용해 merge column을 만들어야 한다.

1. micro_data

In [5]:
micro_data[["tmp1","tmp2", "tmp3"]] = micro_data["회계년도"].str.split("-", n=2, expand=True)
micro_data["tmp1"] = micro_data["tmp1"].str.split("0", n=1, expand=True)[1]
micro_data["merge"] = micro_data["회사명"] + '-' + micro_data["tmp1"] + '-' + micro_data["tmp2"]
micro_data.drop(["산업명"], axis=1).head()

,회사명,회계년도,소속코드,설립일,종업원,당좌자산(*),재고자산(*),비유동자산(*),유동부채(*),비유동부채(*),...,영업활동으로 인한 현금흐름(*),투자활동으로 인한 현금흐름(*),재무활동으로 인한 현금흐름(*),환율변동으로 인한 차이조정,현금의 증가(감소)(*),기업규모명,tmp1,tmp2,tmp3,merge
0,(주)CMG제약,2011-03-01,5.0,2001-08-01,213.0,9917659.0,3104211.0,15477735.0,4969819.0,4069836.0,...,-347445.0,82274.0,0.0,0.0,-265171.0,NaN,11,03,01,(주)CMG제약-11-03
1,(주)CMG제약,2011-06-01,5.0,2001-08-01,213.0,11341653.0,3071427.0,14731279.0,5976935.0,3193612.0,...,-836291.0,-2379507.0,2055075.0,0.0,-1160724.0,NaN,11,06,01,(주)CMG제약-11-06
2,(주)CMG제약,2011-09-01,5.0,2001-08-01,213.0,11407699.0,3352328.0,14717723.0,6389723.0,3071559.0,...,-676550.0,-3250426.0,1965287.0,0.0,-1961689.0,NaN,11,09,01,(주)CMG제약-11-09
3,(주)CMG제약,2011-12-01,5.0,2001-08-01,213.0,12521782.0,2743103.0,12283654.0,7620349.0,1637275.0,...,-2488268.0,-1845332.0,3038758.0,-14.0,-1294857.0,중소기업,11,12,01,(주)CMG제약-11-12
4,(주)CMG제약,2012-03-01,5.0,2001-08-01,213.0,13370740.0,3654331.0,12100895.0,8146944.0,1669899.0,...,-629410.0,-98171.0,1163834.0,-306.0,435947.0,NaN,12,03,01,(주)CMG제약-12-03


2. closing_data

In [6]:
closing_data_369 = closing_data[closing_data["거래월(*)"].isin([3, 6, 9])]
closing_data_12 = closing_data[closing_data["거래월(*)"].isin([12])]

In [7]:
closing_data_369[["tmp1","tmp2"]] = closing_data_369["회계년도"].str.split("-", n=1, expand=True)
closing_data_369["merge"] = closing_data_369["회사명"] + '-' + closing_data_369["tmp2"] + '-' + '0' + closing_data_369["거래월(*)"].astype(int).astype(str)
tmp1 = closing_data_369.drop(["회사명", "회계년도", "거래월(*)", "tmp1","tmp2"],axis=1)
tmp1.head()

c:\users\student\documents\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
c:\users\student\documents\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,산술평균,merge
2,873.0,(주)CMG제약-11-03
5,577.0,(주)CMG제약-11-06
8,993.0,(주)CMG제약-11-09
14,892.0,(주)CMG제약-12-03
17,775.0,(주)CMG제약-12-06


In [8]:
closing_data_12[["tmp1","tmp2"]] = closing_data_12["회계년도"].str.split("-", n=1, expand=True)
closing_data_12["merge"] = closing_data_12["회사명"] + '-' + closing_data_12["tmp2"] + '-' + closing_data_12["거래월(*)"].astype(int).astype(str)
tmp2 = closing_data_12.drop(["회사명", "회계년도", "거래월(*)", "tmp1","tmp2"],axis=1)
tmp2.head()

c:\users\student\documents\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,산술평균,merge
11,775.0,(주)CMG제약-11-12
23,1387.0,(주)CMG제약-12-12
35,923.0,(주)CMG제약-13-12
47,964.0,(주)CMG제약-14-12
59,1987.0,(주)CMG제약-15-12


In [9]:
tmp3 = pd.concat([tmp1, tmp2])

new_micro_data = pd.merge(micro_data, tmp3, how="left")
new_micro_data = new_micro_data.drop(['tmp1','tmp2','tmp3','merge'],axis=1)

### new_micro_data와 macro_data 연결하기

1. new_micro_data에 merge column만들기

In [10]:
year = new_micro_data["회계년도"].str.split('-', n=2, expand=True)[0]
month = new_micro_data["회계년도"].str.split('-', n=2, expand=True)[1]
new_micro_data["merge"] = year + month

2. macro_data에 merge column 만들기

In [11]:
year = macro_data['항목명1'].str.split(' ', n=1, expand=True)[0]
quater = macro_data['항목명1'].str.split(' ', n=1, expand=True)[1]

macro_data['merge'] = 0

macro_data.loc[quater=='1',"merge"] = year[quater=="1"] + '03'
macro_data.loc[quater=='2',"merge"] = year[quater=="2"] + '06'
macro_data.loc[quater=='3',"merge"] = year[quater=="3"] + '09'
macro_data.loc[quater=='4',"merge"] = year[quater=="4"] + '12'

In [12]:
final_data = pd.merge(new_micro_data, macro_data, how="left")

In [13]:
final_data.drop(["merge", "항목명1"], axis=1, inplace=True)

### 코스피지수와 경상수지값 float로 만들기

코스피지수와 경상수지값이 str이므로 int로 바꿔야한다.

In [14]:
final_data[" 코스피지수"] = final_data[" 코스피지수"].str.replace(',',
                                    '').str.replace('\t', '').astype(float)
final_data["  경상수지"] = final_data["  경상수지"].str.replace(",",
                                    '').astype(float)

### 기업규모명 채우기
우리의 데이터의 경우 4분기에만 기업규모명이 있다. 이 마지막 분기를 기준으로 1년 간 이 기업의 규모를 예측할 수 있다고 생각하여, 마지막 분기를 기준으로 기업규모명의 결측치를 채운다.

In [15]:
final_data["기업규모명"] = final_data["기업규모명"].fillna(method="bfill")

### final_data와 stock_data 연결하기

In [16]:
year = final_data["회계년도"].str.split('-', n=2, expand=True)[0]
month = final_data["회계년도"].str.split('-', n=2, expand=True)[1]
final_data["merge"] = final_data["회사명"]+ year + month

In [17]:
year = '20' + stock_data["회계년도"].str.split('-', n=2, expand=True)[1]
month = stock_data["회계년도"].str.split('-', n=2, expand=True)[0]


stock_data['merge'] = 0

stock_data.loc[month=='Mar',"merge"] = stock_data[month=="Mar"]["회사명"] + year[month=="Mar"] + '03'
stock_data.loc[month=='Jun',"merge"] = stock_data[month=="Jun"]["회사명"] + year[month=="Jun"] + '06'
stock_data.loc[month=='Sep',"merge"] = stock_data[month=="Sep"]["회사명"] + year[month=="Sep"] + '09'
stock_data.loc[month=='Dec',"merge"] = stock_data[month=="Dec"]["회사명"] + year[month=="Dec"] + '12'

stock_data.drop(["회사명", "회계년도"], axis=1, inplace=True)

In [18]:
real_final_data = pd.merge(final_data, stock_data, how="left")


In [20]:
real_final_data

,회사명,회계년도,산업명,소속코드,설립일,종업원,당좌자산(*),재고자산(*),비유동자산(*),유동부채(*),...,현금의 증가(감소)(*),기업규모명,산술평균,경제성장률,총저축률,제조업 평균가동률,실업률,코스피지수,경상수지,발행주식수
0,(주)CMG제약,2011-03-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,9917659.0,3104211.0,15477735.0,4969819.0,...,-265171.0,중소기업,873.0,1.0,34.1,81.3,4.2,1939.30,-4253.0,108887244.0
1,(주)CMG제약,2011-06-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,11341653.0,3071427.0,14731279.0,5976935.0,...,-1160724.0,중소기업,577.0,0.6,34.1,80.6,3.4,2142.47,-2470.6,108887244.0
2,(주)CMG제약,2011-09-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,11407699.0,3352328.0,14717723.0,6389723.0,...,-1961689.0,중소기업,993.0,0.7,34.8,80.9,3.1,1880.11,11405.6,108887244.0
3,(주)CMG제약,2011-12-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,12521782.0,2743103.0,12283654.0,7620349.0,...,-1294857.0,중소기업,775.0,0.6,35.2,79.3,2.9,1847.51,11956.2,108887244.0
4,(주)CMG제약,2012-03-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,13370740.0,3654331.0,12100895.0,8146944.0,...,435947.0,중소기업,892.0,0.5,34.3,79.7,3.8,2045.08,-1292.0,108887244.0
5,(주)CMG제약,2012-06-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,11695276.0,2335753.0,11698319.0,7530174.0,...,-345761.0,중소기업,775.0,0.6,34.8,79.5,3.2,1843.47,10940.3,108887244.0
6,(주)CMG제약,2012-09-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,31343658.0,2420524.0,11751541.0,7957865.0,...,19144905.0,중소기업,885.0,0.3,33.7,76.5,3.0,1905.12,16477.8,108887244.0
7,(주)CMG제약,2012-12-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,44118270.0,2955172.0,9915161.0,6665126.0,...,3520645.0,중소기업,1387.0,0.7,33.9,78.3,2.8,1932.90,22664.5,108887244.0
8,(주)CMG제약,2013-03-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,43790630.0,3516371.0,9888824.0,7653796.0,...,-1926670.0,중소기업,1632.0,0.6,34.2,77.3,3.6,2026.49,11705.1,108887244.0
9,(주)CMG제약,2013-06-01,의료용 물질 및 의약품 제조업,5.0,2001-08-01,213.0,42833679.0,3765122.0,10905216.0,7488032.0,...,6319131.0,중소기업,1434.0,1.1,34.5,76.2,3.1,2001.05,21472.9,108887244.0


In [19]:
real_final_data.drop(["merge"], axis=1, inplace=True)

### csv파일 만들기

In [52]:
real_final_data.to_csv("DB_data.csv", header=True, index=False, encoding='ms949')